# 소프트맥스 회귀 구현
---

In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(0)

In [2]:
z = torch.FloatTensor([1, 2, 3])

In [3]:
h = F.softmax(z, dim=0)
print(h)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
h.sum()

tensor(1.)

In [6]:
# 비용함수 구현
z = torch.rand(3, 5, requires_grad=True) # 3x5행렬
print(z)

tensor([[0.5185, 0.6977, 0.8000, 0.1610, 0.2823],
        [0.6816, 0.9152, 0.3971, 0.8742, 0.4194],
        [0.5529, 0.9527, 0.0362, 0.1852, 0.3734]], requires_grad=True)


In [8]:
h = F.softmax(z, dim= 1) # dim=1 두번째 차원에서 softmax 함수를 적용
print(h)

tensor([[0.1996, 0.2387, 0.2645, 0.1396, 0.1576],
        [0.2001, 0.2528, 0.1506, 0.2426, 0.1540],
        [0.2166, 0.3231, 0.1292, 0.1500, 0.1810]], grad_fn=<SoftmaxBackward0>)


In [9]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([1, 2, 3])


In [10]:
y_one_hot = torch.zeros_like(h) # one-hot
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [11]:
print(y.unsqueeze(1)) 

tensor([[1],
        [2],
        [3]])


In [12]:
print(y_one_hot)

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])


In [13]:
cost = (y_one_hot * -torch.log(h)).sum(dim=1).mean()
print(cost)

tensor(1.7410, grad_fn=<MeanBackward0>)


In [14]:
torch.log(F.softmax(z, dim=1))

tensor([[-1.6115, -1.4324, -1.3300, -1.9690, -1.8478],
        [-1.6089, -1.3753, -1.8934, -1.4163, -1.8711],
        [-1.5295, -1.1297, -2.0463, -1.8972, -1.7090]], grad_fn=<LogBackward0>)

In [15]:
F.log_softmax(z, dim=1)

tensor([[-1.6115, -1.4324, -1.3300, -1.9690, -1.8478],
        [-1.6089, -1.3753, -1.8934, -1.4163, -1.8711],
        [-1.5295, -1.1297, -2.0463, -1.8972, -1.7090]],
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
# 다 같은 수식

In [17]:
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.7410, grad_fn=<MeanBackward0>)

In [18]:
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.7410, grad_fn=<MeanBackward0>)

In [19]:
F.nll_loss(F.log_softmax(z, dim=1), y) # Negative Log Likelihood

tensor(1.7410, grad_fn=<NllLossBackward0>)

In [20]:
F.cross_entropy(z, y) # 비용 함수에 소프트맥스 함수까지 포함하고 있음

tensor(1.7410, grad_fn=<NllLossBackward0>)

---

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)

In [39]:
X_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2,2,2,1,1,1,0,0]
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)

In [40]:
print(X_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [41]:
y_train.unsqueeze(1)

tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [1],
        [0],
        [0]])

In [42]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1) ,1)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [43]:
w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([w, b], lr=0.1)

In [44]:
epochs = 1000

for epoch in range(epochs+1) :
    h = F.softmax(X_train.matmul(w) + b, dim= 1)
    cost = (y_one_hot * - torch.log(h)).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(
        epoch, epochs, cost.item()))
    
    
    
    

Epoch    0/1000 Cost : 1.098612
Epoch  100/1000 Cost : 0.761050
Epoch  200/1000 Cost : 0.689991
Epoch  300/1000 Cost : 0.643229
Epoch  400/1000 Cost : 0.604117
Epoch  500/1000 Cost : 0.568255
Epoch  600/1000 Cost : 0.533922
Epoch  700/1000 Cost : 0.500291
Epoch  800/1000 Cost : 0.466908
Epoch  900/1000 Cost : 0.433507
Epoch 1000/1000 Cost : 0.399962


In [45]:
w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([w, b], lr=0.1)

epochs = 1000
for epoch in range(epochs +1) :
    z = X_train.matmul(w) + b
    cost = F.cross_entropy(z, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))
    
    

Epoch    1/1000 Cost: 1.032775
Epoch    2/1000 Cost: 0.985576
Epoch    3/1000 Cost: 0.967064
Epoch    4/1000 Cost: 0.966850
Epoch    5/1000 Cost: 0.985291
Epoch    6/1000 Cost: 1.032398
Epoch    7/1000 Cost: 1.047309
Epoch    8/1000 Cost: 1.091330
Epoch    9/1000 Cost: 1.030258
Epoch   10/1000 Cost: 1.069288
Epoch   11/1000 Cost: 0.996265
Epoch   12/1000 Cost: 1.040883
Epoch   13/1000 Cost: 0.969515
Epoch   14/1000 Cost: 1.016612
Epoch   15/1000 Cost: 0.947535
Epoch   16/1000 Cost: 0.995413
Epoch   17/1000 Cost: 0.929055
Epoch   18/1000 Cost: 0.976886
Epoch   19/1000 Cost: 0.913204
Epoch   20/1000 Cost: 0.960612
Epoch   21/1000 Cost: 0.899367
Epoch   22/1000 Cost: 0.946210
Epoch   23/1000 Cost: 0.887106
Epoch   24/1000 Cost: 0.933357
Epoch   25/1000 Cost: 0.876102
Epoch   26/1000 Cost: 0.921794
Epoch   27/1000 Cost: 0.866122
Epoch   28/1000 Cost: 0.911309
Epoch   29/1000 Cost: 0.856992
Epoch   30/1000 Cost: 0.901738
Epoch   31/1000 Cost: 0.848577
Epoch   32/1000 Cost: 0.892948
Epoch   

Epoch  676/1000 Cost: 0.508326
Epoch  677/1000 Cost: 0.471194
Epoch  678/1000 Cost: 0.507656
Epoch  679/1000 Cost: 0.470650
Epoch  680/1000 Cost: 0.506986
Epoch  681/1000 Cost: 0.470106
Epoch  682/1000 Cost: 0.506316
Epoch  683/1000 Cost: 0.469563
Epoch  684/1000 Cost: 0.505646
Epoch  685/1000 Cost: 0.469020
Epoch  686/1000 Cost: 0.504976
Epoch  687/1000 Cost: 0.468478
Epoch  688/1000 Cost: 0.504307
Epoch  689/1000 Cost: 0.467936
Epoch  690/1000 Cost: 0.503637
Epoch  691/1000 Cost: 0.467395
Epoch  692/1000 Cost: 0.502968
Epoch  693/1000 Cost: 0.466853
Epoch  694/1000 Cost: 0.502298
Epoch  695/1000 Cost: 0.466313
Epoch  696/1000 Cost: 0.501629
Epoch  697/1000 Cost: 0.465772
Epoch  698/1000 Cost: 0.500960
Epoch  699/1000 Cost: 0.465232
Epoch  701/1000 Cost: 0.464692
Epoch  702/1000 Cost: 0.499622
Epoch  703/1000 Cost: 0.464153
Epoch  704/1000 Cost: 0.498953
Epoch  705/1000 Cost: 0.463614
Epoch  706/1000 Cost: 0.498285
Epoch  707/1000 Cost: 0.463076
Epoch  708/1000 Cost: 0.497616
Epoch  7

nn.Module로 구현
---

In [46]:
model = nn.Linear(4, 3) # input_dim = 4, output_dim  =3  // 4개 특성, 3개 분류

In [47]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
epochs = 1000
for epoch in range(epochs + 1) :
    pred = model(X_train)
    cost = F.cross_entropy(pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))
    
    

Epoch    0/1000 Cost: 2.256626
Epoch  100/1000 Cost: 0.643801
Epoch  200/1000 Cost: 0.560924
Epoch  300/1000 Cost: 0.506655
Epoch  400/1000 Cost: 0.462380
Epoch  500/1000 Cost: 0.422981
Epoch  600/1000 Cost: 0.386127
Epoch  700/1000 Cost: 0.350313
Epoch  800/1000 Cost: 0.314385
Epoch  900/1000 Cost: 0.277837
Epoch 1000/1000 Cost: 0.246133


In [50]:
class softmaxClassifierModel(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x) :
        return self.linear(x)

In [51]:
model = softmaxClassifierModel()

In [53]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

epochs = 1000
for epoch in range(epochs + 1) :
    pred = model(X_train)
    cost = F.cross_entropy(pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))
    

Epoch    0/1000 Cost: 1.664998
Epoch  100/1000 Cost: 0.685892
Epoch  200/1000 Cost: 0.605657
Epoch  300/1000 Cost: 0.548294
Epoch  400/1000 Cost: 0.497711
Epoch  500/1000 Cost: 0.449861
Epoch  600/1000 Cost: 0.403203
Epoch  700/1000 Cost: 0.357041
Epoch  800/1000 Cost: 0.311311
Epoch  900/1000 Cost: 0.267972
Epoch 1000/1000 Cost: 0.240508
